In [ ]:
# tests for noise!
using NoiseRobustDifferentiation
# start with given data
time_range = 0.0:0.001:0.1
space_range = 0.0:0.01:1.0
data_array = sol(time_range, space_range)[1]
time_array = collect(time_range)
space_array= collect(space_range)

# initialize grid
g = TimeSpaceGrid1D(time_array, space_array)
# prepare LHS

dt_data_array = copy(data_array)
for i in 1:g.nx
    dt_data_array[:,i] = tvdiff(data_array[:,i], 1, 1e-9; dx=g.dt, scale="small", precond="none")
end

# prepare RHS
# construct relevant variables from highest order spatial derivative to consider
max_derivative_degree = 3
variables_matrix = [data_array]
current_array = copy(data_array)
for j in 1:g.nt
    current_array[j, :] = tvdiff(current_array[j, :], 100, 1e-5, dx=g.dx)            
end
push!(variables_matrix, current_array)
current_array2 = copy(current_array)
for j in 1:g.nt
    current_array2[j, :] = tvdiff(current_array2[j, :], 100, 1e-5, dx=g.dx)            
end
push!(variables_matrix, current_array2)
using Plots
p1 = heatmap(dt_data_array[1:20,1:15], title="ut")
p2 = heatmap(variables_matrix[1][1:20,1:15], title="u")
p3 = heatmap(variables_matrix[2][1:20,1:15], title="ux")
p4 = heatmap(variables_matrix[3][1:20,1:15], title="uxx")
plot(p1, p2, p3, p4, layout = (2,2))


In [ ]:
# comparison for heat analytic
true_sol = zero(data_array)
dt_true_sol = zero(data_array)
d1_true_sol = zero(data_array)
d2_true_sol = zero(data_array)
for (i,t) in enumerate(time_range), (j,x) in enumerate(space_range)
    true_sol[i,j] = exp(- pi^2 * t) * sin(pi*x) + 0.5 * exp(- 3.0^2 * pi^2 * t) * sin(3.0*pi*x) 
    dt_true_sol[i,j] = - pi^2 * exp(- pi^2 * t) * sin(pi*x) - 3.0^2 * 0.5 * pi^2 * exp(- 3.0^2 * pi^2 * t) * sin(3.0*pi*x) 
    d1_true_sol[i,j] = pi * exp(- pi^2 * t) * cos(pi*x) + 1.5 * pi * exp(- 3.0^2 * pi^2 * t) * cos(3.0*pi*x)
    d2_true_sol[i,j] = - pi^2 * exp(- pi^2 * t) * sin(pi*x) - 3.0^2 * 0.5 * pi^2 * exp(- 3.0^2 * pi^2 * t) * sin(3.0*pi*x) 
end
time_range = 0.0:0.001:0.1
space_range = 0.0:0.02:1.0
data_array = true_sol#sol(time_range, space_range)[1]
time_array = collect(time_range)
space_array= collect(space_range)

# initialize grid
g = TimeSpaceGrid1D(time_array, space_array)
# prepare LHS
# make finite difference operator
∂t = TimeDerivative(g) # takes BC and order as arguments
dt_data_array = ∂t(data_array) 
dt_data_array_flat = reshape(dt_data_array, g.N) 

# prepare RHS
# construct relevant variables from highest order spatial derivative to consider
∂x = XDerivative(g; degree=1, order=4) # takes BC and order as arguments
∂x² = XDerivative(g; degree=2, order=2) # takes BC and order as arguments
d1_data_array = ∂x(data_array)
d2_data_array = ∂x(∂x(data_array))
# d2_data_array = ∂x²(data_array)

variables_matrix = [reshape(data_array, g.N),reshape(d1_data_array, g.N),reshape(d2_data_array, g.N)] 

using Plots
p1 = heatmap(((dt_data_array - dt_true_sol)/dt_true_sol)[1:20,1:15], title="ut")
p2 = heatmap((data_array    -    true_sol)[1:20,1:15], title="u")
p3 = heatmap(((d1_data_array - d1_true_sol)/d1_true_sol)[1:20,1:15], title="ux")
p4 = heatmap(((d2_data_array - d2_true_sol)/d2_true_sol)[1:20,1:15], title="uxx")
plot(p1, p2, p3, p4, layout = (2,2))

In [ ]:
# proof that my norm is better
using LinearAlgebra
a, b, c, d = [], [], [], []
for dx in 0.002.*collect(1:1:10), dt in 0.001.*collect(1:1:20)
    # dt = 0.001
    # start with given data
    time_range = 0.0:dt:0.3
    space_range = 0.0:dx:1.0
    data_array = sol(time_range, space_range)[1]
    time_array = collect(time_range)
    space_array= collect(space_range)
    time_len= length(time_array)
    space_len= length(space_array)


    # initialize grid
    g = TimeSpaceGrid1D(time_array, space_array)
    # prepare LHS
    # make finite difference operator
    ∂t = TimeDerivative(g) # takes BC and order as arguments
    dt_data_array = ∂t(data_array) 
    dt_data_array_flat = reshape(dt_data_array, g.N) 

    # prepare RHS
    # construct relevant variables from highest order spatial derivative to consider
    ∂x = XDerivative(g; order=2) # takes BC and order as arguments
    max_derivative_degree = 2
    variables_matrix = GetVariablesMatrix(max_derivative_degree, data_array, g, ∂x)

    # construct basis functions, eg: polynomial basis
    max_poly_degree = 2
    n_variables = max_derivative_degree + 1 # include 0th-derivative i.e. function itself
    MyBasis = PolynomialBasis(max_poly_degree, n_variables, skip_constant = false)

    # theta matrix
    Θ = evaluate(MyBasis, variables_matrix)
    cond_number = cond(Θ, 2) #norm(Θ, 2) * norm(pinv(Θ),2)

    xi,_ = STRidge_cascade(Θ, dt_data_array_flat, MyBasis, λ = 1e2, tol = 0.0, iters = 1, verbose = false, normalize_columns = true)

    push!(a, time_len)
    push!(b, space_len)
    push!(c, cond_number)
    push!(d, norm(Θ * xi - dt_data_array_flat, 2)^2)
   
end

using Plots
N = a.*b
scatter(N, d .*1e3 ./N ./ c, xscale=:log10, yscale=:log10, label="Rudy et. al. norm", xlabel="N", ylabel="2-error / condition number multiplier")
scatter!(N, d .*1e6 ./N ./ c.^2, xscale=:log10, yscale=:log10, label="Modified norm")

In [ ]:
# investigate behaviour at different $\eta$.
to_plot = lambda_sweep(lambda_range, Θ, dt_data_array_flat, MyBasis.poly_vectors, iters=5, cond_number_multiplier=1e-9)
p1 = plot(legend=:bottomleft, xlabel="λ", ylabel="|ξᵢ|", title="η = 1e-9")
for i in 1:length(MyBasis.poly_vectors)
    try
        plot!(to_plot[i,1], abs.(to_plot[i,2]), label=MyBasis.poly_vectors[i], xscale=:log10, yscale=:log10, linewidth=2, marker=:circle, markersize=6, markerstrokewidth=0)
    catch
    end
end

to_plot = lambda_sweep(lambda_range, Θ, dt_data_array_flat, MyBasis.poly_vectors, iters=5, cond_number_multiplier=1e-6)
p2 = plot(legend=:bottomleft, xlabel="λ", title="η = 1e-6")
for i in 1:length(MyBasis.poly_vectors)
    try
        plot!(to_plot[i,1], abs.(to_plot[i,2]), label=MyBasis.poly_vectors[i], xscale=:log10, yscale=:log10, linewidth=2, marker=:circle, markersize=6, markerstrokewidth=0)
    catch
    end
end

to_plot = lambda_sweep(lambda_range, Θ, dt_data_array_flat, MyBasis.poly_vectors, iters=5, cond_number_multiplier=1e-3)
p3 = plot(legend=:bottomleft, xlabel="λ", title="η = 1e-3")
for i in 1:length(MyBasis.poly_vectors)
    try
        plot!(to_plot[i,1], abs.(to_plot[i,2]), label=MyBasis.poly_vectors[i], xscale=:log10, yscale=:log10, linewidth=2, marker=:circle, markersize=6, markerstrokewidth=0)
    catch
    end
end

plot(p1, p2, p3, layout = (1,3), sharey=true)